In [1]:
import pandas as pd

# 파일 경로 설정
book_info_path = r'단행본(도서)정보.csv'
loan_info_path = r'대출정보.csv'


# 파일 읽기 (euc-kr 인코딩)
book_info_df = pd.read_csv(book_info_path, encoding='euc-kr')
loan_info_df = pd.read_csv(loan_info_path, encoding='euc-kr')

# Unnamed 열 제거
book_info_df = book_info_df.loc[:, ~book_info_df.columns.str.contains('^Unnamed')]
loan_info_df = loan_info_df.loc[:, ~loan_info_df.columns.str.contains('^Unnamed')]

# 대출 정보 전처리: 대출일자 추출
loan_info_df['대출일자'] = pd.to_datetime(loan_info_df['대출일시'])

# 대출 요약 정보 생성: 마지막 대출일자와 총 대출 수
loan_summary = loan_info_df.groupby('도서ID').agg(
    마지막대출일자=('대출일자', 'max'),
    총대출수=('대출일자', 'count')
).reset_index()

# 단행본 정보와 대출 요약 정보 병합 (단행본 정보 모든 열 유지)
merged_info = pd.merge(book_info_df, loan_summary, on='도서ID', how='left')

# 대출 기록이 없는 도서 처리
merged_info['마지막대출일자'] = merged_info['마지막대출일자'].fillna('대출 기록 없음')
merged_info['총대출수'] = merged_info['총대출수'].fillna(0).astype(int)

# 결과를 CSV 파일로 저장
output_path = r'통합_정보.csv'
merged_info.to_csv(output_path, index=False, encoding='euc-kr')

# 결과 미리 보기
merged_info.head()


,도서ID,등록일자,수서방법,분류코드,ISBN,서명,저자,출판사,출판년도,소장위치,마지막대출일자,총대출수
0,SS_270312,2017-09-13,학과신청,194,9782020658706,Le courage de la verite :le gouvernement de so...,"Foucault, Michel",Gallimard,2009,4층인문,대출 기록 없음,0
1,SS_233793,2008-10-13,수서정보없음,160,9788995605189,김나지움 논리학입문,"Hegel, Georg Wilhelm Friedrich",용의숲,2008,보존서고,2018-11-26 15:48:04,4
2,SS_233788,2008-10-13,수서정보없음,160,9788995605189,김나지움 논리학입문,"Hegel, Georg Wilhelm Friedrich",용의숲,2008,4층인문,2022-08-23 16:48:14,1
3,SS_075978,2010-09-09,이용자희망,160,9788977757301,선제논리를 향하여,김영정,철학과현실사,2010,4층인문,대출 기록 없음,0
4,SS_297286,2011-06-22,사서선정,160,9788977757301,선제논리를 향하여,김영정,철학과현실사,2010,보존서고,대출 기록 없음,0


In [18]:
import pandas as pd

# 통합_정보.csv 파일 로드
output_path = r'통합_정보.csv'
merged_info = pd.read_csv(output_path, encoding='euc-kr')

# 소장위치별 도서 수 계산
location_counts = merged_info['소장위치'].value_counts()

# 보존서고와 4층인문의 책 수 확인
preserved_books = location_counts.get('보존서고', 0)
fourth_floor_books = location_counts.get('4층인문', 0)

print(f"보존서고에 있는 책: {preserved_books}권")
print(f"4층인문에 있는 책: {fourth_floor_books}권")


보존서고에 있는 책: 138697권
4층인문에 있는 책: 162454권


In [22]:
import pandas as pd

# 통합_정보.csv 파일 로드

output_path = r'통합_정보.csv'
merged_info = pd.read_csv(output_path, encoding='euc-kr')

# '보존서고'에 있는 책 필터링
preserved_books = merged_info[merged_info['소장위치'] == '보존서고']

# '마지막대출일자'가 '대출 기록 없음'이 아닌 경우만 고려
preserved_books['마지막대출일자'] = pd.to_datetime(preserved_books['마지막대출일자'], errors='coerce')

# 2024년 제외
filtered_dates = preserved_books.loc[
    preserved_books['마지막대출일자'].dt.year != 2024
]

# 가장 최근 대출 날짜와 해당 행 찾기
most_recent_loan_date = filtered_dates['마지막대출일자'].max()
most_recent_row = filtered_dates[filtered_dates['마지막대출일자'] == most_recent_loan_date]

# 행 번호 출력
if most_recent_row.empty:
    print("보존서고에 있는 책 중 2024년 이전에 대출 기록이 없습니다.")
else:
    print(f"보존서고에 있는 책 중 2024년 제외 가장 최근에 대출받은 날짜: {most_recent_loan_date.date()}")
    print(f"해당 데이터의 행 번호: {most_recent_row.index.tolist()}")


보존서고에 있는 책 중 2024년 제외 가장 최근에 대출받은 날짜: 2023-12-29
해당 데이터의 행 번호: [57195]


C:\Users\songbh0304\AppData\Local\Temp\ipykernel_15576\1108632819.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preserved_books['마지막대출일자'] = pd.to_datetime(preserved_books['마지막대출일자'], errors='coerce')


In [3]:
import pandas as pd

# 통합_정보.csv 파일 로드
output_path = r'통합_정보.csv'
merged_info = pd.read_csv(output_path, encoding='euc-kr')

# '4층인문'에 있는 책 필터링
fourth_floor_books = merged_info[merged_info['소장위치'] == '4층인문']

# '마지막대출일자'가 '대출 기록 없음'이 아닌 경우만 고려
fourth_floor_books['마지막대출일자'] = pd.to_datetime(fourth_floor_books['마지막대출일자'], errors='coerce')

# 가장 오래된 대출 날짜와 해당 행 찾기
oldest_loan_date = fourth_floor_books['마지막대출일자'].min()
oldest_row = fourth_floor_books[fourth_floor_books['마지막대출일자'] == oldest_loan_date]

# 행 번호 출력
if oldest_row.empty or pd.isna(oldest_loan_date):
    print("4층인문에 있는 책 중 대출 기록이 없습니다.")
else:
    print(f"4층인문에 있는 책 중 가장 대출일자가 오래된 날짜: {oldest_loan_date.date()}")
    print(f"해당 데이터의 행 번호: {oldest_row.index.tolist()}")


/var/folders/2g/9_1xks7j21j8_vjr52g7g8hr0000gn/T/ipykernel_64467/3570931706.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  fourth_floor_books['마지막대출일자'] = pd.to_datetime(fourth_floor_books['마지막대출일자'], errors='coerce')


4층인문에 있는 책 중 가장 대출일자가 오래된 날짜: 2004-11-01
해당 데이터의 행 번호: [202402]


/var/folders/2g/9_1xks7j21j8_vjr52g7g8hr0000gn/T/ipykernel_64467/3570931706.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fourth_floor_books['마지막대출일자'] = pd.to_datetime(fourth_floor_books['마지막대출일자'], errors='coerce')


In [4]:
import pandas as pd

# "통합_정보.csv" 파일 경로 설정
file_path = '통합_정보.csv'

# CSV 파일 로드
merged_info = pd.read_csv(file_path, encoding='euc-kr')

# NA 값을 처리 (groupby를 위한 열에서 누락된 값 방지)
merged_info[['서명', '출판사', '출판년도', '저자']] = merged_info[['서명', '출판사', '출판년도', '저자']].fillna('')

# '마지막대출일자'를 datetime 형식으로 변환
merged_info['마지막대출일자'] = pd.to_datetime(merged_info['마지막대출일자'], errors='coerce')

# 중복된 책 정보를 합침
grouped_books = merged_info.groupby(['서명', '저자', '출판사', '출판년도']).agg(
    총대출수=('총대출수', 'sum'),  # 총 대출 수 합산
    마지막대출일자=('마지막대출일자', 'max'),  # 가장 최근 대출일자 선택
    
    중복책권수=('서명', 'size')  # 중복된 행 수
).reset_index()

# 마지막대출일자를 다시 문자열로 변환 (필요시)
grouped_books['마지막대출일자'] = grouped_books['마지막대출일자'].dt.strftime('%Y-%m-%d').fillna(0)


# 결과 저장 (필요시)
output_path = '통합_정보_합침.csv'
grouped_books.to_csv(output_path, index=False, encoding='euc-kr')

# 연도를 숫자로만 바꾸는 함수 정의
def clean_year(year):
    import re
    match = re.search(r'\d{4}', year)
    return match.group(0) if match else year

# 출판년도 컬럼 변환
grouped_books["출판년도"] = grouped_books["출판년도"].apply(clean_year)

# 결과 미리 보기
grouped_books.head()


/var/folders/2g/9_1xks7j21j8_vjr52g7g8hr0000gn/T/ipykernel_64467/343586763.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_info['마지막대출일자'] = pd.to_datetime(merged_info['마지막대출일자'], errors='coerce')


,서명,저자,출판사,출판년도,총대출수,마지막대출일자,중복책권수
0,"""Das Recht kann nicht ungerecht sein …"" :Beitr...","Li, Wenchao",Franz Steiner Verlag,2015,1,2016-06-07,1
1,"""Der Zusammenhang der Dinge"" :Weltgleichnis un...","Gebhard, Walter",M. Niemeyer,1984,0,0,1
2,"""Ethik und Asthetik sind Eins"" :Beitrage zu Wi...","Lutterfelds, Wilhelm",Peter Lang,2007,1,2017-12-12,1
3,"""God ordained this war"" :sermons on the sectio...","Chesebrough, David B",University of South Carolina Press,1991,0,0,1
4,"""Huis clos"" de Jean-Paul Sartre","Bishop, Thomas",Hachette,1979,2,2017-05-22,1
